### Questions to answer

1. Member vs Casual Total Rides and Average Trip Duration 
2. Member vs Casual Rides per Different Ride Types and Average Trip Duration
3. Member vs Casual Rides Per Month
4. Member vs Casual Rides Per Weekday
5. Member vs Casual Rides Per Hour
6. Reference: Top 3 Most Popular Starting Station among riders
7. Top 10 Member Trip Starting Stations 
    Instead of "WHERE member_casual = 'member'" the statement below was used because it contained a non-printable character which gave it a length of 7
8. Top 10 Member Trip Ending Stations
9. Top 10 Casual Trip Starting Stations
10. Top 10 Casual Trip Ending Stations
11. The Most Popular Starting Station Hourly for Member Riders
12. The Most Popular Ending Station Hourly for Member Riders
13. Combine the Results of the Two Earlier Queries and Sum the Total Trips (Most Popular Member Station Trips Per Hour)
14. Most Popular Casual Station Trips Per Hour

15. Most Popular Casual Routes
16. Most Popular Member Routes
17. Calculate the average latitudes and longitudes for each station and group by stations, member_casual, hour of day


In [1]:
# importing the nessary libraries
import os
import numpy as np
from datetime import datetime
import sqlite3
import pandas as pd


In [2]:
# Create connection to SQLite database
conn = sqlite3.connect('cyclistic_data.db')
cursor = conn.cursor()


In [3]:
# %%
# Load CSV data into SQLite (if not already loaded)
data = pd.read_csv(r'Data\12_Months_data\combined_cyclistic_data.csv')
data.to_sql('cyclistic_data', conn, if_exists='replace', index=False)
conn.commit()


In [4]:
# ============================================================================
# Q1. Member vs Casual Total Rides and Average Trip Duration
# Why: Establishes baseline demand and pace of use between segments.
# ============================================================================
q1 = pd.read_sql_query("""
    SELECT 
        member_casual,
        COUNT(ride_id) AS total_rides,
        AVG(ride_time_min) AS average_trip_duration
    FROM cyclistic_data
    GROUP BY member_casual
""", conn)

print("Q1: Member vs Casual Total Rides and Average Trip Duration")
print(q1)
print()


Q1: Member vs Casual Total Rides and Average Trip Duration
  member_casual  total_rides  average_trip_duration
0        casual      1340478              22.620622
1        member      2400566              12.190921



In [5]:
# ============================================================================
# Q2. Member vs Casual by Ride Type + Avg Duration
# Why: vehicle preferences inform pricing/messaging.
# ============================================================================
q2 = pd.read_sql_query("""
    SELECT 
        member_casual,
        rideable_type,
        COUNT(ride_id) AS total_rides,
        AVG(ride_time_min) AS average_trip_duration
    FROM cyclistic_data
    GROUP BY member_casual, rideable_type
    ORDER BY member_casual, rideable_type
""", conn)

print("Q2: Member vs Casual by Ride Type + Avg Duration")
print(q2)
print()


Q2: Member vs Casual by Ride Type + Avg Duration
  member_casual  rideable_type  total_rides  average_trip_duration
0        casual   classic_bike       717186              28.737248
1        casual  electric_bike       623292              15.582575
2        member   classic_bike      1354193              13.156934
3        member  electric_bike      1046373              10.940729



In [6]:
# ============================================================================
# Q3. Member vs Casual Rides Per Month
# Why: seasonality for planning campaigns.
# ============================================================================
q3 = pd.read_sql_query("""
    SELECT 
        member_casual,
        month,
        COUNT(ride_id) AS total_rides
    FROM cyclistic_data
    GROUP BY member_casual, month
    ORDER BY member_casual, month
""", conn)

print("Q3: Member vs Casual Rides Per Month")
print(q3)
print()


Q3: Member vs Casual Rides Per Month
   member_casual month  total_rides
0         casual   Apr        76324
1         casual   Aug       219136
2         casual   Dec        28038
3         casual   Feb        19481
4         casual   Jan        16946
5         casual   Jul       206460
6         casual   Jun       191120
7         casual   Mar        61053
8         casual   May       124476
9         casual   Nov        68393
10        casual   Oct       158438
11        casual   Sep       170613
12        member   Apr       180626
13        member   Aug       291765
14        member   Dec       102074
15        member   Feb        89410
16        member   Jan        83657
17        member   Jul       283129
18        member   Jun       251485
19        member   Mar       147079
20        member   May       213633
21        member   Nov       176415
22        member   Oct       288460
23        member   Sep       292833



In [7]:
# ============================================================================
# Q4. Member vs Casual Rides Per Weekday
# Why: separate commute vs leisure behavior.
# ============================================================================
q4 = pd.read_sql_query("""
    SELECT 
        member_casual,
        weekday,
        COUNT(ride_id) AS total_rides
    FROM cyclistic_data
    GROUP BY member_casual, weekday
    ORDER BY member_casual, weekday
""", conn)

print("Q4: Member vs Casual Rides Per Weekday")
print(q4)
print()


Q4: Member vs Casual Rides Per Weekday
   member_casual weekday  total_rides
0         casual     Fri       209455
1         casual     Mon       159482
2         casual     Sat       275895
3         casual     Sun       231154
4         casual     Thu       164786
5         casual     Tue       154907
6         casual     Wed       144799
7         member     Fri       346320
8         member     Mon       358507
9         member     Sat       287759
10        member     Sun       260056
11        member     Thu       376852
12        member     Tue       399488
13        member     Wed       371584



In [8]:
# ============================================================================
# Q5. Member vs Casual Rides Per Hour
# Why: identify peak windows for targeting.
# ============================================================================
q5 = pd.read_sql_query("""
    SELECT 
        member_casual,
        hour,
        COUNT(ride_id) AS total_rides
    FROM cyclistic_data
    GROUP BY member_casual, hour
    ORDER BY member_casual, hour
""", conn)

print("Q5: Member vs Casual Rides Per Hour")
print(q5)
print()


Q5: Member vs Casual Rides Per Hour
   member_casual   hour  total_rides
0         casual   1 am        14095
1         casual   1 pm        93242
2         casual  10 am        61332
3         casual  10 pm        38524
4         casual  11 am        78472
5         casual  11 pm        27853
6         casual  12 am        22439
7         casual  12 pm        90821
8         casual   2 am         8899
9         casual   2 pm        97771
10        casual   3 am         4840
11        casual   3 pm       106985
12        casual   4 am         3936
13        casual   4 pm       122275
14        casual   5 am         7259
15        casual   5 pm       128717
16        casual   6 am        18339
17        casual   6 pm       105535
18        casual   7 am        34512
19        casual   7 pm        75581
20        casual   8 am        49485
21        casual   8 pm        54447
22        casual   9 am        49497
23        casual   9 pm        45622
24        member   1 am        10435
25

In [9]:
# ============================================================================
# Q6. Top 3 Most Popular Starting Stations (all riders)
# Why: quick hotspot reference.
# ============================================================================
q6 = pd.read_sql_query("""
    SELECT 
        member_casual,
        start_station_name,
        COUNT(ride_id) AS total_rides
    FROM cyclistic_data
    GROUP BY member_casual, start_station_name
    ORDER BY member_casual, total_rides DESC
""", conn)

print("Q6: Top 3 Most Popular Starting Stations (all riders)")
print(q6)
print()


Q6: Top 3 Most Popular Starting Stations (all riders)
     member_casual                 start_station_name  total_rides
0           casual  DuSable Lake Shore Dr & Monroe St        29082
1           casual            Streeter Dr & Grand Ave        28415
2           casual              Michigan Ave & Oak St        20164
3           casual                          Navy Pier        18533
4           casual                    Millennium Park        17862
...            ...                                ...          ...
3367        member  Public Rack - 10557 S Western Ave            1
3368        member           Plainfield & Irving Park            1
3369        member                      NEW HASTINGS             1
3370        member               Hamlin Ave & 62nd Pl            1
3371        member           Eggleston Ave & 111th St            1

[3372 rows x 3 columns]



In [10]:
# ============================================================================
# Q7. Top 10 Member Trip Starting Stations
# Why: member-focused signage/ops.
# ============================================================================
q7 = pd.read_sql_query("""
    SELECT 
        start_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'member'
    GROUP BY start_station_name
    ORDER BY rides DESC
    LIMIT 10
""", conn)

print("Q7: Top 10 Member Trip Starting Stations")
print(q7)
print()


Q7: Top 10 Member Trip Starting Stations
             start_station_name  rides
0      Kingsbury St & Kinzie St  27753
1  Clinton St & Washington Blvd  22936
2       Clinton St & Madison St  20469
3             Clark St & Elm St  19747
4         Canal St & Madison St  19170
5     Clinton St & Jackson Blvd  17044
6        State St & Chicago Ave  16191
7             Wells St & Elm St  16091
8      University Ave & 57th St  15779
9         Wells St & Concord Ln  15530



In [11]:
# ============================================================================
# Q8. Top 10 Member Trip Ending Stations
# Why: member destination hotspots.
# ============================================================================
q8 = pd.read_sql_query("""
    SELECT 
        end_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'member'
    GROUP BY end_station_name
    ORDER BY rides DESC
    LIMIT 10
""", conn)

print("Q8: Top 10 Member Trip Ending Stations")
print(q8)
print()


Q8: Top 10 Member Trip Ending Stations
               end_station_name  rides
0      Kingsbury St & Kinzie St  28068
1  Clinton St & Washington Blvd  22342
2       Clinton St & Madison St  21034
3             Clark St & Elm St  20182
4         Canal St & Madison St  19550
5     Clinton St & Jackson Blvd  17013
6        State St & Chicago Ave  16702
7             Wells St & Elm St  16266
8         Wells St & Concord Ln  16068
9      University Ave & 57th St  15840



In [12]:
# ============================================================================
# Q9. Top 10 Casual Trip Starting Stations
# Why: prime conversion entry points.
# ============================================================================
q9 = pd.read_sql_query("""
    SELECT 
        start_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'casual'
    GROUP BY start_station_name
    ORDER BY rides DESC
    LIMIT 10
""", conn)

print("Q9: Top 10 Casual Trip Starting Stations")
print(q9)
print()


Q9: Top 10 Casual Trip Starting Stations
                   start_station_name  rides
0   DuSable Lake Shore Dr & Monroe St  29082
1             Streeter Dr & Grand Ave  28415
2               Michigan Ave & Oak St  20164
3                           Navy Pier  18533
4                     Millennium Park  17862
5  DuSable Lake Shore Dr & North Blvd  17415
6                      Shedd Aquarium  16829
7                      Dusable Harbor  14753
8                 Theater on the Lake  14114
9               Michigan Ave & 8th St  10730



In [13]:
# ============================================================================
# Q10. Top 10 Casual Trip Ending Stations
# Why: popular casual destinations.
# ============================================================================
q10 = pd.read_sql_query("""
    SELECT 
        end_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'casual'
    GROUP BY end_station_name
    ORDER BY rides DESC
    LIMIT 10
""", conn)

print("Q10: Top 10 Casual Trip Ending Stations")
print(q10)
print()


Q10: Top 10 Casual Trip Ending Stations
                     end_station_name  rides
0             Streeter Dr & Grand Ave  30337
1   DuSable Lake Shore Dr & Monroe St  27329
2               Michigan Ave & Oak St  20893
3  DuSable Lake Shore Dr & North Blvd  20294
4                     Millennium Park  19241
5                           Navy Pier  19021
6                 Theater on the Lake  15777
7                      Shedd Aquarium  14920
8                      Dusable Harbor  13105
9        Michigan Ave & Washington St   9724



In [14]:
# ============================================================================
# Q11. Most Popular Starting Station HOURLY for Member Riders
# Why: hour-level member hotspots.
# ============================================================================
q11_top = pd.read_sql_query("""
    WITH member_start_hour AS (
        SELECT 
            start_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'member'
        GROUP BY start_station_name, hour
    ),
    ranked_starts AS (
        SELECT 
            start_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM member_start_hour
    )
    SELECT 
        start_station_name,
        hour,
        rides
    FROM ranked_starts
    WHERE rn = 1
    ORDER BY hour
""", conn)

print("Q11: Most Popular Starting Station HOURLY for Member Riders")
print(q11_top)
print()


Q11: Most Popular Starting Station HOURLY for Member Riders
                     start_station_name   hour  rides
0                Halsted St & Roscoe St   1 am    172
1              Kingsbury St & Kinzie St   1 pm   1484
2              Kingsbury St & Kinzie St  10 am   1109
3                 Wells St & Concord Ln  10 pm    505
4              Kingsbury St & Kinzie St  11 am   1239
5                   Wells St & Huron St  11 pm    335
6                Halsted St & Roscoe St  12 am    248
7              Kingsbury St & Kinzie St  12 pm   1439
8               Broadway & Waveland Ave   2 am    133
9              Kingsbury St & Kinzie St   2 pm   1414
10              Broadway & Waveland Ave   3 am     84
11             Kingsbury St & Kinzie St   3 pm   1832
12  DuSable Lake Shore Dr & Belmont Ave   4 am    162
13                Canal St & Madison St   4 pm   3125
14           Indiana Ave & Roosevelt Rd   5 am    473
15         Clinton St & Washington Blvd   5 pm   3807
16         Clinton St 

In [15]:
# ============================================================================
# Q12. Most Popular Ending Station HOURLY for Member Riders
# Why: hour-level member destinations.
# ============================================================================
q12_top = pd.read_sql_query("""
    WITH member_end_hour AS (
        SELECT 
            end_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'member'
        GROUP BY end_station_name, hour
    ),
    ranked_ends AS (
        SELECT 
            end_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM member_end_hour
    )
    SELECT 
        end_station_name,
        hour,
        rides
    FROM ranked_ends
    WHERE rn = 1
    ORDER BY hour
""", conn)

print("Q12: Most Popular Ending Station HOURLY for Member Riders")
print(q12_top)
print()


Q12: Most Popular Ending Station HOURLY for Member Riders
                end_station_name   hour  rides
0            MLK Jr Dr & 29th St   1 am    155
1       University Ave & 57th St   1 pm   1299
2       Kingsbury St & Kinzie St  10 am   1232
3              Clark St & Elm St  10 pm    481
4       Kingsbury St & Kinzie St  11 am   1295
5            Ellis Ave & 60th St  11 pm    381
6            MLK Jr Dr & 29th St  12 am    259
7       Kingsbury St & Kinzie St  12 pm   1316
8            Wells St & Huron St   2 am     62
9       Kingsbury St & Kinzie St   2 pm   1222
10        Marine Dr & Ainslie St   3 am     93
11  Clinton St & Washington Blvd   3 pm   2597
12      Kingsbury St & Kinzie St   4 am    223
13  Clinton St & Washington Blvd   4 pm   3327
14      Kingsbury St & Kinzie St   5 am   1012
15      Kingsbury St & Kinzie St   5 pm   2851
16     LaSalle St & Jackson Blvd   6 am   2245
17             Clark St & Elm St   6 pm   1988
18  Clinton St & Washington Blvd   7 am   3262
19

In [16]:
# ============================================================================
# Q13. Combine Q11 & Q12 and Sum the Total Trips 
# (Most Popular Member Station Trips Per Hour)
# ============================================================================
q13 = pd.read_sql_query("""
    WITH member_start_hour AS (
        SELECT 
            start_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'member'
        GROUP BY start_station_name, hour
    ),
    ranked_starts AS (
        SELECT 
            start_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM member_start_hour
    ),
    top_starts AS (
        SELECT start_station_name, hour, rides
        FROM ranked_starts
        WHERE rn = 1
    ),
    member_end_hour AS (
        SELECT 
            end_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'member'
        GROUP BY end_station_name, hour
    ),
    ranked_ends AS (
        SELECT 
            end_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM member_end_hour
    ),
    top_ends AS (
        SELECT end_station_name, hour, rides
        FROM ranked_ends
        WHERE rn = 1
    )
    SELECT 
        ts.hour,
        ts.start_station_name,
        ts.rides AS rides_start,
        te.end_station_name,
        te.rides AS rides_end,
        ts.rides + COALESCE(te.rides, 0) AS total_rides_top_pairs
    FROM top_starts ts
    FULL OUTER JOIN top_ends te ON ts.hour = te.hour
    ORDER BY ts.hour
""", conn)

print("Q13: Most Popular Member Station Trips Per Hour (Combined)")
print(q13.head(24))
print()


Q13: Most Popular Member Station Trips Per Hour (Combined)
     hour                   start_station_name  rides_start  \
0    1 am               Halsted St & Roscoe St          172   
1    1 pm             Kingsbury St & Kinzie St         1484   
2   10 am             Kingsbury St & Kinzie St         1109   
3   10 pm                Wells St & Concord Ln          505   
4   11 am             Kingsbury St & Kinzie St         1239   
5   11 pm                  Wells St & Huron St          335   
6   12 am               Halsted St & Roscoe St          248   
7   12 pm             Kingsbury St & Kinzie St         1439   
8    2 am              Broadway & Waveland Ave          133   
9    2 pm             Kingsbury St & Kinzie St         1414   
10   3 am              Broadway & Waveland Ave           84   
11   3 pm             Kingsbury St & Kinzie St         1832   
12   4 am  DuSable Lake Shore Dr & Belmont Ave          162   
13   4 pm                Canal St & Madison St         3125

In [17]:
# ============================================================================
# Q14. Most Popular Casual Station Trips Per Hour
# ============================================================================
q14 = pd.read_sql_query("""
    WITH casual_start_hour AS (
        SELECT 
            start_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'casual'
        GROUP BY start_station_name, hour
    ),
    ranked_casual_starts AS (
        SELECT 
            start_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM casual_start_hour
    ),
    casual_start_top AS (
        SELECT start_station_name, hour, rides
        FROM ranked_casual_starts
        WHERE rn = 1
    ),
    casual_end_hour AS (
        SELECT 
            end_station_name,
            hour,
            COUNT(ride_id) AS rides
        FROM cyclistic_data
        WHERE TRIM(LOWER(member_casual)) = 'casual'
        GROUP BY end_station_name, hour
    ),
    ranked_casual_ends AS (
        SELECT 
            end_station_name,
            hour,
            rides,
            ROW_NUMBER() OVER (PARTITION BY hour ORDER BY rides DESC) AS rn
        FROM casual_end_hour
    ),
    casual_end_top AS (
        SELECT end_station_name, hour, rides
        FROM ranked_casual_ends
        WHERE rn = 1
    )
    SELECT 
        cst.hour,
        cst.start_station_name,
        cst.rides AS rides_start,
        cet.end_station_name,
        cet.rides AS rides_end
    FROM casual_start_top cst
    FULL OUTER JOIN casual_end_top cet ON cst.hour = cet.hour
    ORDER BY cst.hour
""", conn)

print("Q14: Most Popular Casual Station Trips Per Hour")
print(q14)
print()


Q14: Most Popular Casual Station Trips Per Hour
     hour                 start_station_name  rides_start  \
0    1 am     California Ave & Milwaukee Ave          144   
1    1 pm  DuSable Lake Shore Dr & Monroe St         3007   
2   10 am  DuSable Lake Shore Dr & Monroe St         1612   
3   10 pm            Streeter Dr & Grand Ave          588   
4   11 am  DuSable Lake Shore Dr & Monroe St         2282   
5   11 pm                     Shedd Aquarium          499   
6   12 am                    Millennium Park          226   
7   12 pm  DuSable Lake Shore Dr & Monroe St         2515   
8    2 am     Sheffield Ave & Wrightwood Ave          137   
9    2 pm  DuSable Lake Shore Dr & Monroe St         3191   
10   3 am            Broadway & Waveland Ave           85   
11   3 pm            Streeter Dr & Grand Ave         3339   
12   4 am       Ashland Ave & Wellington Ave          115   
13   4 pm            Streeter Dr & Grand Ave         3048   
14   5 am                  Clark St &

In [18]:
# ============================================================================
# Q15. Most Popular Casual Routes
# ============================================================================
q15 = pd.read_sql_query("""
    SELECT 
        start_station_name,
        end_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'casual'
    GROUP BY start_station_name, end_station_name
    ORDER BY rides DESC
    LIMIT 20
""", conn)

print("Q15: Most Popular Casual Routes")
print(q15)
print()


Q15: Most Popular Casual Routes
                    start_station_name                    end_station_name  \
0    DuSable Lake Shore Dr & Monroe St   DuSable Lake Shore Dr & Monroe St   
1              Streeter Dr & Grand Ave             Streeter Dr & Grand Ave   
2                            Navy Pier                           Navy Pier   
3                Michigan Ave & Oak St               Michigan Ave & Oak St   
4    DuSable Lake Shore Dr & Monroe St             Streeter Dr & Grand Ave   
5                      Millennium Park                     Millennium Park   
6                       Dusable Harbor                      Dusable Harbor   
7                       Shedd Aquarium   DuSable Lake Shore Dr & Monroe St   
8    DuSable Lake Shore Dr & Monroe St                           Navy Pier   
9                      Montrose Harbor                     Montrose Harbor   
10  DuSable Lake Shore Dr & North Blvd  DuSable Lake Shore Dr & North Blvd   
11   DuSable Lake Shore Dr & Mon

In [19]:
# ============================================================================
# Q16. Most Popular Member Routes
# ============================================================================
q16 = pd.read_sql_query("""
    SELECT 
        start_station_name,
        end_station_name,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    WHERE TRIM(LOWER(member_casual)) = 'member'
    GROUP BY start_station_name, end_station_name
    ORDER BY rides DESC
    LIMIT 20
""", conn)

print("Q16: Most Popular Member Routes")
print(q16)
print()


Q16: Most Popular Member Routes
          start_station_name              end_station_name  rides
0        Ellis Ave & 60th St           Ellis Ave & 55th St   3799
1   University Ave & 57th St           Ellis Ave & 60th St   3678
2        Ellis Ave & 60th St      University Ave & 57th St   3582
3        Ellis Ave & 55th St           Ellis Ave & 60th St   3553
4      Calumet Ave & 33rd St            State St & 33rd St   2445
5         State St & 33rd St         Calumet Ave & 33rd St   2341
6   Blackstone Ave & 59th St      University Ave & 57th St   1735
7   Loomis St & Lexington St           Morgan St & Polk St   1688
8   University Ave & 57th St         Kimbark Ave & 53rd St   1617
9   University Ave & 57th St       Lake Park Ave & 56th St   1610
10     Kimbark Ave & 53rd St      University Ave & 57th St   1590
11  University Ave & 57th St      Blackstone Ave & 59th St   1582
12       Morgan St & Polk St      Loomis St & Lexington St   1499
13       Ellis Ave & 55th St         Kimbark

In [20]:
# ============================================================================
# Q17a. Average latitudes and longitudes for each station 
# grouped by member_casual and hour of day (Starting Stations)
# ============================================================================
q17_start = pd.read_sql_query("""
    SELECT 
        start_station_name,
        member_casual,
        hour,
        AVG(start_lat) AS avg_lat,
        AVG(start_lng) AS avg_lng,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    GROUP BY start_station_name, member_casual, hour
    ORDER BY start_station_name, member_casual, hour
""", conn)

print("Q17a: Average lat/lng for Starting Stations")
print(q17_start.head(20))
print()


Q17a: Average lat/lng for Starting Stations
     start_station_name member_casual   hour    avg_lat    avg_lng  rides
0   2112 W Peterson Ave        casual   1 pm  41.991178 -87.683593     23
1   2112 W Peterson Ave        casual  10 am  41.991178 -87.683593      8
2   2112 W Peterson Ave        casual  10 pm  41.991178 -87.683593      3
3   2112 W Peterson Ave        casual  11 am  41.991178 -87.683593     10
4   2112 W Peterson Ave        casual  11 pm  41.991178 -87.683593      3
5   2112 W Peterson Ave        casual  12 am  41.991178 -87.683593      4
6   2112 W Peterson Ave        casual  12 pm  41.991178 -87.683593     15
7   2112 W Peterson Ave        casual   2 pm  41.991178 -87.683593     14
8   2112 W Peterson Ave        casual   3 pm  41.991178 -87.683593     19
9   2112 W Peterson Ave        casual   4 pm  41.991178 -87.683593     14
10  2112 W Peterson Ave        casual   5 pm  41.991176 -87.683596     17
11  2112 W Peterson Ave        casual   6 pm  41.991178 -87.683593  

In [21]:
# ============================================================================
# Q17b. Average latitudes and longitudes for each station 
# grouped by member_casual and hour of day (Ending Stations)
# ============================================================================
q17_end = pd.read_sql_query("""
    SELECT 
        end_station_name,
        member_casual,
        hour,
        AVG(end_lat) AS avg_lat,
        AVG(end_lng) AS avg_lng,
        COUNT(ride_id) AS rides
    FROM cyclistic_data
    GROUP BY end_station_name, member_casual, hour
    ORDER BY end_station_name, member_casual, hour
""", conn)

print("Q17b: Average lat/lng for Ending Stations")
print(q17_end.head(20))
print()


Q17b: Average lat/lng for Ending Stations
       end_station_name member_casual   hour    avg_lat    avg_lng  rides
0   2112 W Peterson Ave        casual   1 am  41.991178 -87.683593      5
1   2112 W Peterson Ave        casual   1 pm  41.991178 -87.683593     11
2   2112 W Peterson Ave        casual  10 am  41.991178 -87.683593     12
3   2112 W Peterson Ave        casual  10 pm  41.991178 -87.683593      4
4   2112 W Peterson Ave        casual  11 am  41.991178 -87.683593      9
5   2112 W Peterson Ave        casual  11 pm  41.991178 -87.683593      3
6   2112 W Peterson Ave        casual  12 am  41.991178 -87.683593      1
7   2112 W Peterson Ave        casual  12 pm  41.991178 -87.683593     22
8   2112 W Peterson Ave        casual   2 pm  41.991178 -87.683593     12
9   2112 W Peterson Ave        casual   3 pm  41.991178 -87.683593     28
10  2112 W Peterson Ave        casual   4 pm  41.991178 -87.683593     31
11  2112 W Peterson Ave        casual   5 pm  41.991178 -87.683593    

In [22]:
# Create a holidays table with all 2024-2025 federal holidays
holidays_data = [
    ('2024-09-02', 'Labor Day'),
    ('2024-10-14', 'Columbus Day / Indigenous Peoples\' Day'),
    ('2024-11-11', 'Veterans Day'),
    ('2024-11-28', 'Thanksgiving Day'),
    ('2024-12-25', 'Christmas Day'),
    ('2025-01-01', 'New Year\'s Day'),
    ('2025-01-20', 'Martin Luther King Jr. Day'),
    ('2025-02-17', 'Presidents\' Day / Washington\'s Birthday'),
    ('2025-05-26', 'Memorial Day'),
    ('2025-06-19', 'Juneteenth National Independence Day'),
    ('2025-07-04', 'Independence Day'),
    ('2025-09-01', 'Labor Day'),
]

# Create holidays table
cursor.execute('''
    DROP TABLE IF EXISTS holidays
''')

cursor.execute('''
    CREATE TABLE holidays (
        holiday_date TEXT PRIMARY KEY,
        holiday_name TEXT
    )
''')

cursor.executemany('INSERT INTO holidays VALUES (?, ?)', holidays_data)
conn.commit()

print("Holidays table created with {} holidays".format(len(holidays_data)))
print()


Holidays table created with 12 holidays



In [23]:
# ============================================================================
# QUERY 1: Total Rides on Holidays - Members vs Casual
# ============================================================================
q_holiday_total = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        cd.member_casual,
        COUNT(cd.ride_id) AS total_rides,
        AVG(cd.ride_time_min) AS avg_trip_duration_min,
        MIN(cd.ride_time_min) AS min_trip_duration,
        MAX(cd.ride_time_min) AS max_trip_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name, cd.member_casual
    ORDER BY h.holiday_date, cd.member_casual
""", conn)

print("=" * 80)
print("QUERY 1: Total Rides on Holidays - Members vs Casual")
print("=" * 80)
print(q_holiday_total)
print()


QUERY 1: Total Rides on Holidays - Members vs Casual
   holiday_date                             holiday_name member_casual  \
0    2024-10-14   Columbus Day / Indigenous Peoples' Day        casual   
1    2024-10-14   Columbus Day / Indigenous Peoples' Day        member   
2    2024-11-11                             Veterans Day        casual   
3    2024-11-11                             Veterans Day        member   
4    2024-11-28                         Thanksgiving Day        casual   
5    2024-11-28                         Thanksgiving Day        member   
6    2024-12-25                            Christmas Day        casual   
7    2024-12-25                            Christmas Day        member   
8    2025-01-01                           New Year's Day        casual   
9    2025-01-01                           New Year's Day        member   
10   2025-01-20               Martin Luther King Jr. Day        casual   
11   2025-01-20               Martin Luther King Jr. Day   

In [24]:
# ============================================================================
# QUERY 2: Hourly Breakdown of Holiday Rides
# ============================================================================
q_holiday_hourly = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        cd.hour,
        cd.member_casual,
        COUNT(cd.ride_id) AS rides,
        AVG(cd.ride_time_min) AS avg_trip_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name, cd.hour, cd.member_casual
    ORDER BY h.holiday_date, cd.hour, cd.member_casual
""", conn)

print("=" * 80)
print("QUERY 2: Hourly Breakdown of Holiday Rides")
print("=" * 80)
print(q_holiday_hourly)
print()


QUERY 2: Hourly Breakdown of Holiday Rides
    holiday_date                            holiday_name   hour member_casual  \
0     2024-10-14  Columbus Day / Indigenous Peoples' Day   1 am        casual   
1     2024-10-14  Columbus Day / Indigenous Peoples' Day   1 am        member   
2     2024-10-14  Columbus Day / Indigenous Peoples' Day   1 pm        casual   
3     2024-10-14  Columbus Day / Indigenous Peoples' Day   1 pm        member   
4     2024-10-14  Columbus Day / Indigenous Peoples' Day  10 am        casual   
..           ...                                     ...    ...           ...   
521   2025-09-01                               Labor Day   8 pm        member   
522   2025-09-01                               Labor Day   9 am        casual   
523   2025-09-01                               Labor Day   9 am        member   
524   2025-09-01                               Labor Day   9 pm        casual   
525   2025-09-01                               Labor Day   9 pm   

In [25]:
# ============================================================================
# QUERY 3: Ride Type Preference on Holidays - Members vs Casual
# ============================================================================
q_holiday_ridetype = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        cd.member_casual,
        cd.rideable_type,
        COUNT(cd.ride_id) AS rides,
        AVG(cd.ride_time_min) AS avg_trip_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name, cd.member_casual, cd.rideable_type
    ORDER BY h.holiday_date, cd.member_casual, cd.rideable_type
""", conn)

print("=" * 80)
print("QUERY 3: Ride Type Preference on Holidays - Members vs Casual")
print("=" * 80)
print(q_holiday_ridetype)
print()


QUERY 3: Ride Type Preference on Holidays - Members vs Casual
   holiday_date                             holiday_name member_casual  \
0    2024-10-14   Columbus Day / Indigenous Peoples' Day        casual   
1    2024-10-14   Columbus Day / Indigenous Peoples' Day        casual   
2    2024-10-14   Columbus Day / Indigenous Peoples' Day        member   
3    2024-10-14   Columbus Day / Indigenous Peoples' Day        member   
4    2024-11-11                             Veterans Day        casual   
5    2024-11-11                             Veterans Day        casual   
6    2024-11-11                             Veterans Day        member   
7    2024-11-11                             Veterans Day        member   
8    2024-11-28                         Thanksgiving Day        casual   
9    2024-11-28                         Thanksgiving Day        casual   
10   2024-11-28                         Thanksgiving Day        member   
11   2024-11-28                         Thanksgivi

In [26]:
# ============================================================================
# QUERY 4: Top 5 Starting Stations on Each Holiday
# ============================================================================
q_holiday_start_stations = pd.read_sql_query("""
    WITH holiday_starts AS (
        SELECT 
            h.holiday_date,
            h.holiday_name,
            cd.member_casual,
            cd.start_station_name,
            COUNT(cd.ride_id) AS rides,
            ROW_NUMBER() OVER (PARTITION BY h.holiday_date, cd.member_casual 
                              ORDER BY COUNT(cd.ride_id) DESC) AS rnk
        FROM cyclistic_data cd
        INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
        GROUP BY h.holiday_date, h.holiday_name, cd.member_casual, cd.start_station_name
    )
    SELECT 
        holiday_date,
        holiday_name,
        member_casual,
        start_station_name,
        rides
    FROM holiday_starts
    WHERE rnk <= 5
    ORDER BY holiday_date, member_casual, rnk
""", conn)

print("=" * 80)
print("QUERY 4: Top 5 Starting Stations on Each Holiday")
print("=" * 80)
print(q_holiday_start_stations)
print()


QUERY 4: Top 5 Starting Stations on Each Holiday
    holiday_date                            holiday_name member_casual  \
0     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
1     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
2     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
3     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
4     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
..           ...                                     ...           ...   
105   2025-09-01                               Labor Day        member   
106   2025-09-01                               Labor Day        member   
107   2025-09-01                               Labor Day        member   
108   2025-09-01                               Labor Day        member   
109   2025-09-01                               Labor Day        member   

                     start_station_name  rides  
0     DuSable

In [27]:
# ============================================================================
# QUERY 5: Top 5 Ending Stations on Each Holiday
# ============================================================================
q_holiday_end_stations = pd.read_sql_query("""
    WITH holiday_ends AS (
        SELECT 
            h.holiday_date,
            h.holiday_name,
            cd.member_casual,
            cd.end_station_name,
            COUNT(cd.ride_id) AS rides,
            ROW_NUMBER() OVER (PARTITION BY h.holiday_date, cd.member_casual 
                              ORDER BY COUNT(cd.ride_id) DESC) AS rnk
        FROM cyclistic_data cd
        INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
        GROUP BY h.holiday_date, h.holiday_name, cd.member_casual, cd.end_station_name
    )
    SELECT 
        holiday_date,
        holiday_name,
        member_casual,
        end_station_name,
        rides
    FROM holiday_ends
    WHERE rnk <= 5
    ORDER BY holiday_date, member_casual, rnk
""", conn)

print("=" * 80)
print("QUERY 5: Top 5 Ending Stations on Each Holiday")
print("=" * 80)
print(q_holiday_end_stations)
print()


QUERY 5: Top 5 Ending Stations on Each Holiday
    holiday_date                            holiday_name member_casual  \
0     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
1     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
2     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
3     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
4     2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
..           ...                                     ...           ...   
105   2025-09-01                               Labor Day        member   
106   2025-09-01                               Labor Day        member   
107   2025-09-01                               Labor Day        member   
108   2025-09-01                               Labor Day        member   
109   2025-09-01                               Labor Day        member   

                       end_station_name  rides  
0              

In [28]:
# ============================================================================
# QUERY 6: Comparison Summary - Holiday vs Average Day
# ============================================================================
q_holiday_comparison = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        cd.member_casual,
        COUNT(cd.ride_id) AS holiday_rides,
        ROUND(AVG(cd.ride_time_min), 2) AS holiday_avg_duration,
        (
            SELECT ROUND(AVG(ride_time_min), 2)
            FROM cyclistic_data
            WHERE member_casual = cd.member_casual
            AND strftime('%w', started_at) NOT IN 
                (SELECT CAST(strftime('%w', DATE(holiday_date)) AS TEXT) FROM holidays)
        ) AS avg_day_avg_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name, cd.member_casual
    ORDER BY h.holiday_date, cd.member_casual
""", conn)

print("=" * 80)
print("QUERY 6: Holiday Rides vs Average Day Comparison")
print("=" * 80)
print(q_holiday_comparison)
print()


QUERY 6: Holiday Rides vs Average Day Comparison
   holiday_date                             holiday_name member_casual  \
0    2024-10-14   Columbus Day / Indigenous Peoples' Day        casual   
1    2024-10-14   Columbus Day / Indigenous Peoples' Day        member   
2    2024-11-11                             Veterans Day        casual   
3    2024-11-11                             Veterans Day        member   
4    2024-11-28                         Thanksgiving Day        casual   
5    2024-11-28                         Thanksgiving Day        member   
6    2024-12-25                            Christmas Day        casual   
7    2024-12-25                            Christmas Day        member   
8    2025-01-01                           New Year's Day        casual   
9    2025-01-01                           New Year's Day        member   
10   2025-01-20               Martin Luther King Jr. Day        casual   
11   2025-01-20               Martin Luther King Jr. Day       

In [29]:
# ============================================================================
# QUERY 7: Most Popular Routes on Holidays
# ============================================================================
q_holiday_routes = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        cd.member_casual,
        cd.start_station_name,
        cd.end_station_name,
        COUNT(cd.ride_id) AS rides,
        ROUND(AVG(cd.ride_time_min), 2) AS avg_trip_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name, cd.member_casual, 
             cd.start_station_name, cd.end_station_name
    ORDER BY h.holiday_date, cd.member_casual, rides DESC
    LIMIT 50
""", conn)

print("=" * 80)
print("QUERY 7: Most Popular Routes on Holidays (Top 50)")
print("=" * 80)
print(q_holiday_routes)
print()


QUERY 7: Most Popular Routes on Holidays (Top 50)
   holiday_date                            holiday_name member_casual  \
0    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
1    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
2    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
3    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
4    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
5    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
6    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
7    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
8    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
9    2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
10   2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
11   2024-10-14  Columbus Day / Indigenous Peoples' Day        casual   
1

In [30]:
# ============================================================================
# QUERY 8: Statistical Summary by Holiday
# ============================================================================
q_holiday_stats = pd.read_sql_query("""
    SELECT 
        h.holiday_date,
        h.holiday_name,
        COUNT(DISTINCT cd.ride_id) AS total_holiday_rides,
        SUM(CASE WHEN cd.member_casual = 'member' THEN 1 ELSE 0 END) AS member_rides,
        SUM(CASE WHEN cd.member_casual = 'casual' THEN 1 ELSE 0 END) AS casual_rides,
        ROUND(100.0 * SUM(CASE WHEN cd.member_casual = 'member' THEN 1 ELSE 0 END) / 
              COUNT(DISTINCT cd.ride_id), 2) AS member_percentage,
        ROUND(100.0 * SUM(CASE WHEN cd.member_casual = 'casual' THEN 1 ELSE 0 END) / 
              COUNT(DISTINCT cd.ride_id), 2) AS casual_percentage,
        ROUND(AVG(CASE WHEN cd.member_casual = 'member' THEN cd.ride_time_min END), 2) 
            AS avg_member_duration,
        ROUND(AVG(CASE WHEN cd.member_casual = 'casual' THEN cd.ride_time_min END), 2) 
            AS avg_casual_duration
    FROM cyclistic_data cd
    INNER JOIN holidays h ON DATE(cd.started_at) = h.holiday_date
    GROUP BY h.holiday_date, h.holiday_name
    ORDER BY h.holiday_date
""", conn)

print("=" * 80)
print("QUERY 8: Statistical Summary by Holiday")
print("=" * 80)
print(q_holiday_stats)
print()


QUERY 8: Statistical Summary by Holiday
   holiday_date                             holiday_name  total_holiday_rides  \
0    2024-10-14   Columbus Day / Indigenous Peoples' Day                11945   
1    2024-11-11                             Veterans Day                10559   
2    2024-11-28                         Thanksgiving Day                 1942   
3    2024-12-25                            Christmas Day                 1188   
4    2025-01-01                           New Year's Day                 2437   
5    2025-01-20               Martin Luther King Jr. Day                 1241   
6    2025-02-17  Presidents' Day / Washington's Birthday                 1789   
7    2025-05-26                             Memorial Day                10771   
8    2025-06-19     Juneteenth National Independence Day                17380   
9    2025-07-04                         Independence Day                15316   
10   2025-09-01                                Labor Day             

In [31]:
# %%
# Close the connection
conn.close()
